In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from imports import *
from torch_imports import *
from torchtext import vocab, data, datasets

In [3]:
from pt_models import *
from dataset_pt import *

### IMDB test

In [123]:
def postproc(arr,vocab,train): 
    return [1 if o==1 else 0 for o in arr]

In [124]:
# Approach 1:
# set up fields
TEXT = data.Field(lower=True, fix_length=500)
LABEL = data.Field(sequential=False, postprocessing=postproc)

# make splits for data
trn, test = datasets.IMDB.splits(TEXT, LABEL, root='tmp/')

In [125]:
# print information about the data
print('trn.fields', trn.fields)
print('len(trn)', len(trn))
print('vars(trn[0])', vars(trn[0]))

trn.fields {'text': <torchtext.data.field.Field object at 0x7f34cd1571d0>, 'label': <torchtext.data.field.Field object at 0x7f34cd157160>}
len(trn) 25000
vars(trn[0]) {'text': ['i', 'remember', 'when', 'i', 'first', 'saw', 'this', 'short,', 'i', 'was', 'really', 'laughing', 'so', 'hard,', 'that', 'like', 'with', 'a', 'lot', 'of', 'other', 'films', 'that', 'i', 'have', 'seen,', 'no', 'sound', 'came', 'out!', 'curly', 'is', 'really', 'great', 'at', '"singing"', 'opera', 'in', 'this', 'one,', 'i', 'am', 'surprised', 'that', 'he', 'did', 'not', 'consider', 'a', 'career', 'as', 'a', 'professional', 'singer,', 'because', 'he', 'was', 'really', 'good!', '<br', '/><br', '/>if', 'you', 'noticed,', 'this', 'was', 'filmed', 'near', 'the', 'end', 'of', "curly's", 'career', 'as', 'a', 'stooge,', 'you', 'could', 'really', 'tell', 'he', 'had', 'changed,', 'because', 'he', 'had', 'lost', 'weight', 'and', 'was', 'thinner,', 'his', 'voice', 'was', 'deepening,', 'his', 'face', 'was', 'getting', 'lined', 

In [126]:
vocab_size=5000

In [127]:
# build the vocabulary
TEXT.build_vocab(trn, vectors='glove.6B.100d', max_size=vocab_size)
LABEL.build_vocab(trn)

# print vocab information
print('len(TEXT.vocab)', len(TEXT.vocab))
print('TEXT.vocab.vectors.size()', TEXT.vocab.vectors.size())

loading vectors from .vector_cache/glove.6B.100d.pt
len(TEXT.vocab) 5002
TEXT.vocab.vectors.size() torch.Size([5002, 100])


In [128]:
LABEL.vocab.itos

['<unk>', 'neg', 'pos']

In [129]:
TEXT.vocab.vectors


 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0382 -0.2449  0.7281  ...  -0.1459  0.8278  0.2706
          ...             ⋱             ...          
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
-0.0626  0.9136  0.4275  ...  -0.6858  0.5875  0.3384
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
[torch.FloatTensor of size 5002x100]

In [130]:
# make iterator for splits
trn_iter, test_iter = data.BucketIterator.splits((trn, test), batch_size=64, repeat=False)

In [131]:
# print batch information
batch = next(iter(trn_iter))

In [132]:
print(batch.text)
print(batch.label)

Variable containing:
    2     9    21  ...     12     2    48
 3106   236  2808  ...      7   998   668
    7    12   659  ...      3   679   553
       ...          ⋱          ...       
    1     1     1  ...      1     1     1
    1     1     1  ...      1     1     1
    1     1     1  ...      1     1     1
[torch.cuda.LongTensor of size 500x64 (GPU 0)]

Variable containing:
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 0
 1
 1
 1
 0
 1
 1
[torch.cuda.LongTensor of size 64 (GPU 0)]



In [133]:
TEXT.vocab.vectors[:10,:10]


 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
-0.0382 -0.2449  0.7281 -0.3996  0.0832  0.0440 -0.3914  0.3344 -0.5755  0.0875
-0.2709  0.0440 -0.0203 -0.1740  0.6444  0.7121  0.3551  0.4714 -0.2964  0.5443
-0.0720  0.2313  0.0237 -0.5064  0.3392  0.1959 -0.3294  0.1836 -0.1806  0.2896
-0.1529 -0.2428  0.8984  0.1700  0.5352  0.4878 -0.5883 -0.1798 -1.3581  0.4254
-0.1897  0.0500  0.1908 -0.0492 -0.0897  0.2101 -0.5495  0.0984 -0.2014  0.3424
-0.5426  0.4148  1.0322 -0.4024  0.4669  0.2182 -0.0749  0.4733  0.0810 -0.2208
 0.0857 -0.2220  0.1657  0.1337  0.3824  0.3540  0.0129  0.2246 -0.4382  0.5016
-0.0465  0.6197  0.5665 -0.4658 -1.1890  0.4460  0.0660  0.3191  0.1468 -0.2212
[torch.FloatTensor of size 10x10]

In [134]:
len(TEXT.vocab.freqs)

251637

In [135]:
LABEL.vocab.stoi

defaultdict(<function torchtext.vocab.Vocab.__init__.<locals>.<lambda>>,
            {'neg': 1, 'pos': 2})

In [136]:
pad=TEXT.vocab.stoi['<pad>']

In [137]:
class TextDataLoader():
    def __init__(self, src, x_fld, y_fld):
        self.src,self.x_fld,self.y_fld = src,x_fld,y_fld
        
    def __len__(self): return len(self.src)
    
    def __iter__(self): 
        self.src_iter = iter(self.src)
        return self
        
    def __next__(self): 
        try:
            b = next(self.src_iter)
            return getattr(b, self.x_fld), getattr(b, self.y_fld)
        except StopIteration: 
            raise StopIteration

In [138]:
def create_td(it): return TextDataLoader(it, 'text', 'label')

In [139]:
md = ModelData('.', create_td(trn_iter), create_td(test_iter))

In [140]:
x,y=next(iter(md.trn_dl))

In [141]:
nf=100
n_emb=32
nv=len(TEXT.vocab.itos)

class SimpleLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(nv, n_emb, padding_idx=pad)#, max_norm=1., norm_type=2)
        self.emb.weight.data.uniform_(-0.05,0.05)
        self.l1 = nn.Linear(500*n_emb, nf)
        self.l2 = nn.Linear(nf, 2)

    def forward(self, x):
        ns,bs = x.size()
        x = self.emb(x)
        x = x.transpose(0,1).contiguous().view(bs,-1)
        x = F.relu(self.l1(x))
        x = F.dropout(x, 0.7)
        return self.l2(x)

In [142]:
m = SimpleLinear().cuda()
opt = optim.Adam(m.parameters(), 1e-3)

In [143]:
fit(m, md, 2, F.cross_entropy, opt, metrics=[accuracy])

/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.382181  0.338946  0.854867]


[ 0.260172  0.32782   0.860294]



### LSTM

In [144]:
TEXT = data.Field(lower=True)
LABEL = data.Field(sequential=False, postprocessing=postproc)
trn, test = datasets.IMDB.splits(TEXT, LABEL, root='tmp/')

In [145]:
TEXT.build_vocab(trn, vectors='glove.6B.100d', max_size=vocab_size)
LABEL.build_vocab(trn)

loading vectors from .vector_cache/glove.6B.100d.pt


In [146]:
trn_iter, test_iter = data.BucketIterator.splits((trn, test), batch_size=64, repeat=False)

In [147]:
md = ModelData('.', create_td(trn_iter), create_td(test_iter))

In [157]:
hs=100
n_emb=32

class SimpleLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(len(TEXT.vocab.itos), n_emb, padding_idx=pad)
        self.emb.weight.data.uniform_(-0.05,0.05)
        self.rnn = nn.LSTM(n_emb, hs, num_layers=1, dropout=0.2)
        self.out = nn.Linear(hs, 2)
        
    def forward(self, x):
        bs = x.size(1)
        x = self.emb(x)
        c0 = Variable(torch.zeros(1,bs,hs)).cuda()
        h0 = Variable(torch.zeros(1,bs,hs)).cuda()
        outp,_ = self.rnn(x, (h0,c0))
        return self.out(outp[-1])

In [160]:
m = SimpleLSTM().cuda()
opt = optim.Adam(m.parameters(), 1e-2)

In [162]:
fit(m, md, 2, F.cross_entropy, opt, metrics=[accuracy])

/home/jhoward/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:13: DeprecationWarning: generator 'Iterator.__iter__' raised StopIteration


[ 0.260364  0.299499  0.873649]


[ 0.220552  0.325146  0.868175]



### Translation test

In [52]:
import re, spacy

In [53]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')



    Only loading the 'de' tokenizer.



    Only loading the 'en' tokenizer.



In [54]:
url = re.compile('(<url>.*</url>)')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(url.sub('@URL@', text))]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]

DE = data.Field(tokenize=tokenize_de)
EN = data.Field(tokenize=tokenize_en)

From https://wit3.fbk.eu/archive/2016-01//texts/de/en/de-en.tgz

In [56]:
path='/data/jhoward/github/deeplearning/nbs/'

In [58]:
train, val = datasets.TranslationDataset.splits(
    path=path+'tmp/de-en/', train='train.tags.de-en',
    validation='IWSLT16.TED.tst2013.de-en', exts=('.de', '.en'),
    fields=(DE, EN))

print(train.fields)
print(len(train))
print(vars(train[0]))
print(vars(train[100]))

{'src': <torchtext.data.field.Field object at 0x7f7260609048>, 'trg': <torchtext.data.field.Field object at 0x7f7260609128>}
209772
{'src': ['@URL@'], 'trg': ['@URL@']}
{'src': ['BL', ':', 'Diese', 'Würmer', 'haben', 'kein', 'Verdauungssystem', '.', 'Sie', 'haben', 'keinen', 'Mund', '.'], 'trg': ['BL', ':', 'These', 'worms', 'have', 'no', 'digestive', 'system', '.', 'They', 'have', 'no', 'mouth', '.']}


In [59]:
DE.build_vocab(train.src, min_freq=3)
EN.build_vocab(train.trg, max_size=50000)

In [61]:
train_iter, val_iter = data.BucketIterator.splits(
    (train, val), batch_size=3, device=0)

print(DE.vocab.freqs.most_common(10))
print(len(DE.vocab.freqs))
print(EN.vocab.freqs.most_common(10))
print(len(EN.vocab.freqs))

[(',', 293485), ('.', 205869), ('die', 85250), ('und', 77141), ('der', 56257), ('ist', 51461), ('das', 45803), ('zu', 44547), ('in', 44314), ('ich', 39040)]
136046
[(',', 248240), ('.', 196796), ('the', 155955), ('to', 97498), ('of', 91549), ('and', 84765), ('a', 82235), ('that', 69796), ('I', 63840), ('in', 58086)]
65325


In [64]:
batch = next(iter(train_iter))
print(batch.src)
print(batch.trg)

Variable containing:
    22   1004    104
    24      6     38
    84    754     81
   262   4097    250
     2     95     41
    19   1610    459
  7448     47    202
  1387   3758  24275
  2424   1422  10394
    17      7    816
     0      2      2
  1070     19      8
     7    827     66
     2   1001     11
   737      2     94
    11   1117    124
    44   2107      2
  6154      5    157
     2     11     59
   145    221      7
    12     63    166
    15  13079   4817
  1164      2      2
    53     20     14
     2    259    167
   190      2      9
    41     19    598
     2     15      5
    30     17     94
    12   6790    182
  5708   2729    479
   137    663      9
  2511      3    228
 12106      1      3
   691      1      1
  1164      1      1
    53      1      1
     3      1      1
[torch.cuda.LongTensor of size 38x3 (GPU 0)]

Variable containing:
    17    300     94
    51      6     12
    28      4     67
    18    186    400
  1654   5148      4
     9   